# Bayesian optimization on a sphere manifold

This notebooks illustrates the application of Bayesian optimization on a sphere manifold. 
To run it, you need to have botorch installed.


In [1]:
import numpy as np
import random
import torch
import gpytorch
import botorch

In [2]:
import geometric_kernels.torch
from geometric_kernels.spaces.hypersphere import Hypersphere
from geometric_kernels.kernels.geometric_kernels import MaternKarhunenLoeveKernel
from geometric_kernels.frontends.pytorch.gpytorch import GPytorchGeometricKernel

INFO: Using numpy backend


We first set the numpy and pytorch seeds for reproducibility.

In [3]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

We initialize a sphere manifold $\mathcal{S}^2$.

In [15]:
dimension = 3
hypersphere = Hypersphere(dim=dimension-1)

Define the function to optimize. 

Here we use the Ackley function (see, e.g., https://www.sfu.ca/~ssurjano/ackley.html).
The function is defined on the tangent space of the base point $(1, 0, 0, ...)$ and projected to the manifold via the exponential map. The value of the function is therefore computed by projecting the point on the manifold to the tangent space of the base point and by computing the value of the Ackley function in this Euclidean space.

In [5]:
def ackley_function_sphere(x):
    # Data to numpy
    torch_type = x.dtype
    x = x.cpu().detach().numpy()
    if np.ndim(x) < 2:
        x = x[None]

    # Dimension of the manifold
    dimension = x.shape[-1]

    # Projection in tangent space of the mean.
    # The base is fixed at (1, 0, 0, ...) for simplicity. Therefore, the tangent plane is aligned with the axis x.
    # The first coordinate of x_proj is always 0, so that vectors in the tangent space can be expressed in a dim-1
    # dimensional space by simply ignoring the first coordinate.
    base = np.zeros((1, dimension))
    base[0, 0] = 1.
    x_proj = hypersphere.to_tangent(x, base)[0]

    # Remove first dim
    x_proj_red = x_proj[1:]
    reduced_dimension = dimension-1

    # Ackley function parameters
    a = 20
    b = 0.2
    c = 2*np.pi

    # Ackley function
    aexp_term = -a * np.exp(-b * np.sqrt(np.sum(x_proj_red**2) / reduced_dimension))
    expcos_term = - np.exp( np.sum(np.cos(c*x_proj_red) / reduced_dimension))
    y = aexp_term + expcos_term + a + np.exp(1.)

    return torch.tensor(y[None, None], dtype=torch_type)


## Initialization

Generate 5 random data on the sphere to be used as initial points for the BO.

In [21]:
nb_data_init = 5

x_data = torch.tensor(hypersphere.random_point(nb_data_init))
y_data = torch.zeros(nb_data_init, dtype=torch.float64)
for n in range(nb_data_init):
    y_data[n] = ackley_function_sphere(x_data[n])

print('Inputs:', x_data)
print('Outputs: ', y_data)

Inputs: tensor([[-0.0514, -0.8964,  0.4402],
        [ 0.3695, -0.8517, -0.3715],
        [ 0.9320, -0.2240, -0.2850],
        [-0.8520, -0.0264,  0.5228],
        [ 0.8480,  0.5186,  0.1093]], dtype=torch.float64)
Outputs:  tensor([4.4177, 4.2276, 2.7450, 3.1472, 3.2666], dtype=torch.float64)


## Definition of the BO surrogate model
Here we use a Gaussian process with a sphere Matérn kernel. 

We first define the kernel.

In [22]:
_TRUNCATION_LEVEL = 10
base_kernel = GPytorchGeometricKernel(MaternKarhunenLoeveKernel(hypersphere, _TRUNCATION_LEVEL))
kernel = gpytorch.kernels.ScaleKernel(base_kernel,
                                      outputscale_prior=gpytorch.priors.torch_priors.GammaPrior(2.0, 0.15))

The number of eigenfunctions requested does not lead to complete levels of spherical harmonics. We have thus increased the number to 16, which includes all spherical harmonics up to degree 4 (excl.)
The number of eigenfunctions requested does not lead to complete levels of spherical harmonics. We have thus increased the number to 16, which includes all spherical harmonics up to degree 4 (excl.)


We then define the likelihood of the Gaussian process.

In [8]:
noise_prior = gpytorch.priors.torch_priors.GammaPrior(1.1, 0.05)
noise_prior_mode = (noise_prior.concentration - 1) / noise_prior.rate
lik_fct = gpytorch.likelihoods.gaussian_likelihood.GaussianLikelihood(noise_prior=noise_prior,
                                                                      noise_constraint=
                                                                      gpytorch.constraints.GreaterThan(1e-8),
                                                                      initial_value=noise_prior_mode)

We finally initialize the GP model, as well as the marginal likelihood function that will be used to optimize its parameters.

In [9]:
model = botorch.models.SingleTaskGP(x_data, y_data[:, None], covar_module=kernel, likelihood=lik_fct)
mll_fct = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)

## Bayesian optimization loop

In [11]:
new_best_f, index = y_data.min(0)
best_x = [x_data[index]]
best_f = [new_best_f]

We define the bounds for the optimization of the acquisition function, as well as the constraints that must be satisfied by candidate points on the sphere.

In [19]:
bounds = torch.stack([-torch.ones(dimension, dtype=torch.float64), torch.ones(dimension, dtype=torch.float64)])

def upper_constraint(x):
    return 1.0 - torch.linalg.norm(x, dim=-1)

def lower_constraint(x):
    return torch.linalg.norm(x, dim=-1) - 1.0

In [20]:
n_iters = 25
for iteration in range(n_iters):
    # Fit GP model
    botorch.fit_gpytorch_model(mll=mll_fct)

    # Define the acquisition function
    acq_fct = botorch.acquisition.ExpectedImprovement(model=model, best_f=best_f[-1], maximize=False)
    
    # Initial conditions to optimize the acquisition function
    batch_initial_conditions = torch.tensor(hypersphere.random_point(100))
    batch_initial_conditions /= torch.linalg.norm(batch_initial_conditions, dim=-1)[:, None]


    # Get new candidate
    new_x = botorch.optim.optimize_acqf(acq_fct, bounds=bounds, q=1, num_restarts=5,
                                        nonlinear_inequality_constraints=[upper_constraint, lower_constraint],
                                        batch_initial_conditions=batch_initial_conditions[:, None])

    # Get new observation
    new_y = test_function(new_x)[0].to(device)

    # Update training points
    x_data = torch.cat((x_data, new_x))
    y_data = torch.cat((y_data, new_y))

    # Update best observation
    new_best_f, index = y_data.min(0)
    best_x.append(x_data[index])
    best_f.append(new_best_f)

    # Update the model
    model.set_train_data(x_data, y_data, strict=False)  # strict False necessary to add datapoints

    print("Iteration " + str(iteration) + "\t Best f " + str(new_best_f.item()))

ValueError: Einstein sum subscript 'nki' does not contain the correct number of indices for operand 1.

In [ ]:
x_eval = x_data.cpu().numpy()
y_eval = y_data.cpu().numpy()[:, None]
best_x_np = np.array([x.cpu().detach().numpy() for x in best_x])
best_f_np = np.array([f.cpu().detach().numpy() for f in best_f])[:, None]